In [1]:
!pip install image_classifiers
#!pip install keras==3.1.1 tensorflow==2.16.1
import numpy as np
import tensorflow as tf
from keras.applications import ResNet50V2
from keras.datasets import cifar100
from keras import Sequential, Input
from keras.layers import Dense, Dropout, RandomFlip, RandomTranslation, RandomRotation,RandomBrightness, RandomContrast, RandomZoom, GlobalAveragePooling2D
from keras.applications.resnet_v2 import preprocess_input
from keras.models import Model
from classification_models.keras import Classifiers
from keras.optimizers import SGD
from keras.activations import linear
from keras.utils import Progbar
from keras.backend import clear_session
from tensorflow.nn import softmax_cross_entropy_with_logits
import os

2024-04-30 15:33:23.756030: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-30 15:33:23.830566: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-30 15:33:24.983820: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
n_epoch = 50
batch_size = 64
taux_validation = 0.1
num_classes = 100
n_images = 50000 # Pour l'entrainement, et 10000 pour le test

In [3]:
validation_size = int(n_images * taux_validation)
train_size = n_images - validation_size

In [4]:
!mc cp s3/afeldmann/projet_cnam/modele_enseignant.keras /home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras
model_enseignant = Sequential([
    Input((224,224,3)),
    ResNet50V2(include_top=False, weights='imagenet', pooling="avg"),
    Dropout(0.25),
    Dense(256, activation="sigmoid", kernel_regularizer = tf.keras.regularizers.L1(0.001)),
    Dropout(0.5),
    Dense(num_classes, activation="softmax", kernel_regularizer = tf.keras.regularizers.L2(0.001))
])
# Keras 3.1.1 est buggé et le chargement direct ne marche pas ici, même si les poids sont bien enregistrés
model_enseignant.load_weights("/home/onyxia/work/projet_distillation_cnam/sauvegardes/modele_enseignant.keras")

model_enseignant.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

...nant.keras: 135.73 MiB / 135.73 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 106.69 MiB/s 1s

2024-04-30 15:33:32.706232: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13290 MB memory:  -> device: 0, name: NVIDIA A2, pci bus id: 0000:ca:00.0, compute capability: 8.6


In [5]:
def preprocessing(image, label):
    image = tf.image.resize(image, (224, 224))
    label = tf.squeeze(tf.one_hot(label, depth = num_classes), axis = 0)
    return  image, label

augmentation_donnees_keras = Sequential([
    RandomFlip("horizontal"),
    RandomTranslation(0.2,0.2),
    RandomRotation(0.2),
    RandomZoom(0.2),
    RandomContrast(0.2),
    RandomBrightness(0.2,value_range=(0,1))
])

def augmentation_donnees(image, label):
    return augmentation_donnees_keras(image/255.0, training = True)*255.0, label

def preprocess_resnet(image, label):
    return preprocess_input(image), label

def train_val_split(train_dataset, validation_size):
    X_train, y_train = train_dataset
    return (X_train[:train_size,...], y_train[:train_size,...]), (X_train[train_size:,...], y_train[train_size:,...])


def load_cifar_train_val():
    train_dataset, _ = cifar100.load_data()
    
    train_dataset, validation_dataset = train_val_split(train_dataset, validation_size)
    
    validation_dataset = tf.data.Dataset.from_tensor_slices(validation_dataset).map(preprocessing).batch(batch_size).map(preprocess_resnet).cache().prefetch(tf.data.AUTOTUNE)
    train_dataset = tf.data.Dataset.from_tensor_slices(train_dataset).map(preprocessing).cache().repeat().shuffle(train_size).batch(batch_size).map(augmentation_donnees, num_parallel_calls = tf.data.AUTOTUNE).map(preprocess_resnet, num_parallel_calls = tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)
    return train_dataset, validation_dataset

In [6]:
def get_modele_logits(modele):
    config = modele.layers[-1].get_config()
    config['activation'] = linear
    config['name'] = 'logits'
    res = Model(inputs=modele.inputs, outputs=[Dense(**config)(modele.layers[-2].output)])
    res.layers[-1].set_weights([x.numpy() for x in modele.layers[-1].weights])
    res.compile(metrics=['accuracy'])
    return res

def get_regularisation(model):
    return [(getattr(layer,reg),layer.kernel) for layer in model.layers for reg in ["kernel_regularizer", "bias_regularizer"] if hasattr(layer, reg) and getattr(layer,reg) is not None]

@tf.function
def perte_regularisation(regularisation):
    return tf.add_n([tf.reduce_sum(reg(kernel)) for reg, kernel in regularisation])

@tf.function
def compte_bons(x,y):
    return tf.reduce_sum(tf.cast(tf.equal(tf.argmax(x, axis = 1), tf.argmax(y, axis = 1)), tf.float32))

@tf.function
def softmax(logits, temp):
    expo = tf.exp(logits / temp)
    return expo / tf.reduce_sum(expo, axis = 1, keepdims=True)

@tf.function
def ce(x, y_logits, temp):
    return softmax_cross_entropy_with_logits(x, y_logits / temp) * temp**2

def init_csv_log(fichier):
    with open(fichier,'w') as file:
        file.write("epoch, accuracy_etu, accuracy_dis, val_accuracy_etu, val_accuracy_dis\n")
def append_csv_log(fichier, epoch, accuracy_etu, accuracy_dis, val_accuracy_etu, val_accuracy_dis):
    with open(fichier,'a') as file:
        file.write(f"{epoch:d},{accuracy_etu:.2f},{accuracy_dis:.2f},{val_accuracy_etu:.2f},{val_accuracy_dis:.2f}\n")

def forward_backward_pass_impl(train_dataset_iter, etudiant_logit_model, alpha, temp, optim, regularisation_etudiant):
    X_batch, y_batch, enseignant_estim_softmax = next(train_dataset_iter)
    with tf.GradientTape() as tape:
        etudiant_estim_logit = etudiant_logit_model(X_batch, training = True)
        perte = alpha * softmax_cross_entropy_with_logits(y_batch,etudiant_estim_logit) + (1-alpha) * ce(enseignant_estim_softmax,etudiant_estim_logit, temp) + perte_regularisation(regularisation_etudiant)
    grads = tape.gradient(perte, etudiant_logit_model.trainable_variables)
    optim.apply_gradients(zip(grads, etudiant_logit_model.trainable_variables))
    return compte_bons(etudiant_estim_logit,y_batch), compte_bons(etudiant_estim_logit, enseignant_estim_softmax)

@tf.function
def val_accuracies(etudiant, enseignant, validation_dataset):
    val_bons_etu, val_bons_dis = 0.0, 0.0
    for X_batch, y_batch in validation_dataset:
        etu_prev = etudiant(X_batch, training = False)
        ens_prev = enseignant(X_batch, training = False)
        val_bons_etu += compte_bons(etu_prev, y_batch)
        val_bons_dis += compte_bons(etu_prev, ens_prev)
    nb_obs_val = tf.cast((validation_size//batch_size) * batch_size, tf.float32)
    val_accuracy_etu = val_bons_etu / nb_obs_val
    val_accuracy_dis = val_bons_dis / nb_obs_val
    return val_accuracy_etu, val_accuracy_dis

def distillateur_kl(etudiant, enseignant, train_dataset, validation_dataset, temp, nom_modele, n_epoch, alpha):
    etudiant_logit_model = get_modele_logits(etudiant)
    enseignant_logit_model = get_modele_logits(enseignant)
    optim = SGD(learning_rate=0.001)
    init_csv_log(f"sauvegardes/{nom_modele}_logs.csv")
    print("C'est parti pour la distillation !\n")
    val_accuracy_etu_max, val_accuracy_dis_max  = 0, 0
    train_dataset_iter = iter(
        train_dataset
        .map(lambda images, label: (images, label, softmax(enseignant_logit_model(images, training = False), temp)), num_parallel_calls = tf.data.AUTOTUNE)
        .prefetch(tf.data.AUTOTUNE)
    )
    regularisation_etudiant = get_regularisation(etudiant)
    forward_backward_pass = tf.function(forward_backward_pass_impl)
    # La tf.function ne peut être que locale car son graphe dépend d'étudiant_logit_model et sinon Tensorflow renvoie une erreur à deux applications successives
    for epoch in range(n_epoch):
        print(f"Époque {epoch + 1} / {n_epoch}")
        n_batch = train_size//batch_size
        barre_progression = Progbar(n_batch, stateful_metrics = ["acc (etu, train)", "acc (dis, train)"])
        bons_epoque_etu, bons_epoque_dis = 0, 0
        for i in range(n_batch):
            bons_etu, bons_dis = forward_backward_pass(train_dataset_iter, etudiant_logit_model, alpha, temp, optim, regularisation_etudiant)
            bons_epoque_etu += bons_etu.numpy()
            bons_epoque_dis += bons_dis.numpy()
            n_observ = (i+1) * batch_size
            accuracy_etu, accuracy_dis = bons_epoque_etu / n_observ, bons_epoque_dis / n_observ
            barre_progression.update(i + 1, values = [("acc (etu, train)", accuracy_etu), ("acc (dis, train)", accuracy_dis)])
        val_accuracy_etu, val_accuracy_dis = val_accuracies(etudiant, enseignant, validation_dataset)
        if val_accuracy_etu > val_accuracy_etu_max:
            val_accuracy_etu_max = val_accuracy_etu
            etudiant.save(f"sauvegardes/{nom_modele}_checkpoint.keras")
        if val_accuracy_dis > val_accuracy_dis_max:
            val_accuracy_dis_max = val_accuracy_dis
        if epoch + 1 == 40:
            optim.learning_rate.assign(0.0001)
        if epoch + 1 in [10,20,50]:
            print(f"---> Epoque {epoch + 1:d} - Max val accuracy -> etu : {val_accuracy_etu_max:.4f} | dis : {val_accuracy_dis_max:.4f}")
        append_csv_log(f"sauvegardes/{nom_modele}_logs.csv", epoch, accuracy_etu, accuracy_dis, val_accuracy_etu, val_accuracy_dis)
        print(f"Accuracy (etu, val) : {val_accuracy_etu:.4f} | Accuracy (dis, val) : {val_accuracy_dis:.4f}")

In [7]:
def distillation_resnet18(temp, alpha):
    clear_session()
    train_dataset, validation_dataset = load_cifar_train_val()
    modele = new_modele_resnet()
    nom_modele =  f"model_etudiant_t{temp:d}_a{int(alpha*100):d}"
    distillateur_kl(modele, model_enseignant, train_dataset, validation_dataset, temp, nom_modele, n_epoch, alpha)
    wd = os.getcwd()
    os.system(f"cp {wd}/sauvegardes/{nom_modele}_checkpoint.keras {wd}/sauvegardes/{nom_modele}.keras")
    os.system(f"mc cp {wd}/sauvegardes/{nom_modele}.keras s3/afeldmann/projet_cnam/{nom_modele}.keras")
    os.system(f"mc cp {wd}/sauvegardes/{nom_modele}_logs.csv s3/afeldmann/projet_cnam/{nom_modele}_logs.csv")

In [8]:
def ResNet18():
    resnet18, preprocess_input = Classifiers.get('resnet18')
    resnet = resnet18((224, 224, 3), weights='imagenet', include_top=False)
    resnet_output = GlobalAveragePooling2D()(resnet.output)
    resnet = Model(inputs=resnet.input, outputs=resnet_output)
    return resnet

def new_modele_resnet():
    model = Sequential([
        Input((224,224,3)),
        ResNet18(),
        Dropout(0.25),
        Dense(256, activation="sigmoid", kernel_regularizer = tf.keras.regularizers.L1(0.001)),
        Dropout(0.5),
        Dense(num_classes, activation="softmax", kernel_regularizer = tf.keras.regularizers.L2(0.001))
    ])
    model.compile(metrics=['accuracy'])
    return model

In [9]:
distillation_resnet18(1,1) # témoin

C'est parti pour la distillation !

Époque 1 / 50


2024-04-30 15:34:12.313199: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8900


703/703 ━━━━━━━━━━━━━━━━━━━━ 470s 610ms/step - acc (etu, train): 0.1012 - acc (dis, train): 0.1033
Accuracy (etu, val) : 0.1012 | Accuracy (dis, val) : 0.1004
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 359s 511ms/step - acc (etu, train): 0.2716 - acc (dis, train): 0.2790
Accuracy (etu, val) : 0.3393 | Accuracy (dis, val) : 0.3458
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 361s 513ms/step - acc (etu, train): 0.3798 - acc (dis, train): 0.3899
Accuracy (etu, val) : 0.3938 | Accuracy (dis, val) : 0.4014
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 357s 508ms/step - acc (etu, train): 0.4350 - acc (dis, train): 0.4475
Accuracy (etu, val) : 0.4565 | Accuracy (dis, val) : 0.4633
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 359s 510ms/step - acc (etu, train): 0.4823 - acc (dis, train): 0.4952
Accuracy (etu, val) : 0.4828 | Accuracy (dis, val) : 0.4874
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 366s 521ms/step - acc (etu, train): 0.5044 - acc (dis, train): 0.5170
Accuracy (etu, val) : 0.5086 | Accur

In [10]:
distillation_resnet18(1,0)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 472s 642ms/step - acc (etu, train): 0.1069 - acc (dis, train): 0.1091
Accuracy (etu, val) : 0.1092 | Accuracy (dis, val) : 0.1108
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 395s 563ms/step - acc (etu, train): 0.2881 - acc (dis, train): 0.2977
Accuracy (etu, val) : 0.3548 | Accuracy (dis, val) : 0.3624
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 381s 542ms/step - acc (etu, train): 0.3904 - acc (dis, train): 0.4012
Accuracy (etu, val) : 0.4287 | Accuracy (dis, val) : 0.4369
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 375s 533ms/step - acc (etu, train): 0.4487 - acc (dis, train): 0.4625
Accuracy (etu, val) : 0.4677 | Accuracy (dis, val) : 0.4768
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 373s 531ms/step - acc (etu, train): 0.4872 - acc (dis, train): 0.5000
Accuracy (etu, val) : 0.4998 | Accuracy (dis, val) : 0.5072
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 364s 518ms/step - acc (etu, train): 0.5162 - acc (dis, t

In [11]:
distillation_resnet18(3,0)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 462s 631ms/step - acc (etu, train): 0.0722 - acc (dis, train): 0.0747
Accuracy (etu, val) : 0.0875 | Accuracy (dis, val) : 0.0873
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 384s 547ms/step - acc (etu, train): 0.2040 - acc (dis, train): 0.2089
Accuracy (etu, val) : 0.2564 | Accuracy (dis, val) : 0.2574
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 360s 512ms/step - acc (etu, train): 0.3063 - acc (dis, train): 0.3141
Accuracy (etu, val) : 0.3403 | Accuracy (dis, val) : 0.3444
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 339s 483ms/step - acc (etu, train): 0.3805 - acc (dis, train): 0.3916
Accuracy (etu, val) : 0.3998 | Accuracy (dis, val) : 0.4042
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 336s 479ms/step - acc (etu, train): 0.4371 - acc (dis, train): 0.4497
Accuracy (etu, val) : 0.4387 | Accuracy (dis, val) : 0.4465
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 340s 483ms/step - acc (etu, train): 0.4749 - acc (dis, t

In [12]:
distillation_resnet18(8,0)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 450s 613ms/step - acc (etu, train): 0.0364 - acc (dis, train): 0.0361
Accuracy (etu, val) : 0.0611 | Accuracy (dis, val) : 0.0609
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 362s 514ms/step - acc (etu, train): 0.0909 - acc (dis, train): 0.0915
Accuracy (etu, val) : 0.1731 | Accuracy (dis, val) : 0.1777
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 361s 514ms/step - acc (etu, train): 0.1418 - acc (dis, train): 0.1467
Accuracy (etu, val) : 0.2119 | Accuracy (dis, val) : 0.2179
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 351s 500ms/step - acc (etu, train): 0.1910 - acc (dis, train): 0.1958
Accuracy (etu, val) : 0.2524 | Accuracy (dis, val) : 0.2556
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 346s 492ms/step - acc (etu, train): 0.2429 - acc (dis, train): 0.2494
Accuracy (etu, val) : 0.2810 | Accuracy (dis, val) : 0.2875
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 341s 485ms/step - acc (etu, train): 0.2807 - acc (dis, t

In [13]:
distillation_resnet18(1,0.1)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 477s 652ms/step - acc (etu, train): 0.1043 - acc (dis, train): 0.1059
Accuracy (etu, val) : 0.0877 | Accuracy (dis, val) : 0.0891
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 386s 549ms/step - acc (etu, train): 0.2878 - acc (dis, train): 0.2944
Accuracy (etu, val) : 0.3564 | Accuracy (dis, val) : 0.3604
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 384s 547ms/step - acc (etu, train): 0.3874 - acc (dis, train): 0.3996
Accuracy (etu, val) : 0.4307 | Accuracy (dis, val) : 0.4397
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 383s 545ms/step - acc (etu, train): 0.4477 - acc (dis, train): 0.4607
Accuracy (etu, val) : 0.4762 | Accuracy (dis, val) : 0.4880
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 377s 536ms/step - acc (etu, train): 0.4881 - acc (dis, train): 0.5028
Accuracy (etu, val) : 0.5120 | Accuracy (dis, val) : 0.5228
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 376s 535ms/step - acc (etu, train): 0.5201 - acc (dis, t

In [14]:
distillation_resnet18(3,0.1)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 458s 614ms/step - acc (etu, train): 0.0727 - acc (dis, train): 0.0755
Accuracy (etu, val) : 0.0791 | Accuracy (dis, val) : 0.0807
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 379s 539ms/step - acc (etu, train): 0.2148 - acc (dis, train): 0.2205
Accuracy (etu, val) : 0.2881 | Accuracy (dis, val) : 0.2991
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 379s 539ms/step - acc (etu, train): 0.3274 - acc (dis, train): 0.3358
Accuracy (etu, val) : 0.3892 | Accuracy (dis, val) : 0.3984
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 382s 543ms/step - acc (etu, train): 0.3996 - acc (dis, train): 0.4109
Accuracy (etu, val) : 0.4533 | Accuracy (dis, val) : 0.4647
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 382s 543ms/step - acc (etu, train): 0.4532 - acc (dis, train): 0.4659
Accuracy (etu, val) : 0.4748 | Accuracy (dis, val) : 0.4850
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 373s 531ms/step - acc (etu, train): 0.4876 - acc (dis, t

In [15]:
distillation_resnet18(8,0.1)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 453s 615ms/step - acc (etu, train): 0.0425 - acc (dis, train): 0.0424
Accuracy (etu, val) : 0.0571 | Accuracy (dis, val) : 0.0571
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 373s 530ms/step - acc (etu, train): 0.1233 - acc (dis, train): 0.1253
Accuracy (etu, val) : 0.1607 | Accuracy (dis, val) : 0.1659
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 375s 533ms/step - acc (etu, train): 0.1987 - acc (dis, train): 0.2036
Accuracy (etu, val) : 0.2302 | Accuracy (dis, val) : 0.2364
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 374s 532ms/step - acc (etu, train): 0.2602 - acc (dis, train): 0.2674
Accuracy (etu, val) : 0.2985 | Accuracy (dis, val) : 0.3059
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 375s 534ms/step - acc (etu, train): 0.3198 - acc (dis, train): 0.3290
Accuracy (etu, val) : 0.3474 | Accuracy (dis, val) : 0.3544
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 375s 533ms/step - acc (etu, train): 0.3646 - acc (dis, t

In [16]:
distillation_resnet18(1,0.25)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 414s 556ms/step - acc (etu, train): 0.1041 - acc (dis, train): 0.1058
Accuracy (etu, val) : 0.0825 | Accuracy (dis, val) : 0.0841
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 351s 500ms/step - acc (etu, train): 0.2859 - acc (dis, train): 0.2918
Accuracy (etu, val) : 0.3261 | Accuracy (dis, val) : 0.3337
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 381s 542ms/step - acc (etu, train): 0.3862 - acc (dis, train): 0.3965
Accuracy (etu, val) : 0.3894 | Accuracy (dis, val) : 0.4018
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 378s 538ms/step - acc (etu, train): 0.4444 - acc (dis, train): 0.4574
Accuracy (etu, val) : 0.4629 | Accuracy (dis, val) : 0.4748
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 376s 535ms/step - acc (etu, train): 0.4898 - acc (dis, train): 0.5050
Accuracy (etu, val) : 0.4946 | Accuracy (dis, val) : 0.5044
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 367s 522ms/step - acc (etu, train): 0.5184 - acc (dis, t

In [17]:
distillation_resnet18(3,0.25)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 469s 640ms/step - acc (etu, train): 0.0760 - acc (dis, train): 0.0775
Accuracy (etu, val) : 0.0635 | Accuracy (dis, val) : 0.0639
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 372s 530ms/step - acc (etu, train): 0.2215 - acc (dis, train): 0.2270
Accuracy (etu, val) : 0.3049 | Accuracy (dis, val) : 0.3113
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 372s 530ms/step - acc (etu, train): 0.3286 - acc (dis, train): 0.3368
Accuracy (etu, val) : 0.3602 | Accuracy (dis, val) : 0.3642
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 378s 538ms/step - acc (etu, train): 0.4043 - acc (dis, train): 0.4149
Accuracy (etu, val) : 0.4331 | Accuracy (dis, val) : 0.4415
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 368s 523ms/step - acc (etu, train): 0.4522 - acc (dis, train): 0.4658
Accuracy (etu, val) : 0.4836 | Accuracy (dis, val) : 0.4936
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 369s 525ms/step - acc (etu, train): 0.4925 - acc (dis, t

In [18]:
distillation_resnet18(8,0.25)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 471s 633ms/step - acc (etu, train): 0.0541 - acc (dis, train): 0.0547
Accuracy (etu, val) : 0.0579 | Accuracy (dis, val) : 0.0585
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 374s 533ms/step - acc (etu, train): 0.1564 - acc (dis, train): 0.1598
Accuracy (etu, val) : 0.2400 | Accuracy (dis, val) : 0.2444
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 373s 531ms/step - acc (etu, train): 0.2503 - acc (dis, train): 0.2561
Accuracy (etu, val) : 0.2979 | Accuracy (dis, val) : 0.3033
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 374s 533ms/step - acc (etu, train): 0.3289 - acc (dis, train): 0.3370
Accuracy (etu, val) : 0.3696 | Accuracy (dis, val) : 0.3778
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 370s 526ms/step - acc (etu, train): 0.3842 - acc (dis, train): 0.3947
Accuracy (etu, val) : 0.4065 | Accuracy (dis, val) : 0.4159
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 374s 532ms/step - acc (etu, train): 0.4331 - acc (dis, t

In [19]:
distillation_resnet18(1,0.5)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 475s 643ms/step - acc (etu, train): 0.1047 - acc (dis, train): 0.1077
Accuracy (etu, val) : 0.1124 | Accuracy (dis, val) : 0.1124
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 383s 545ms/step - acc (etu, train): 0.2826 - acc (dis, train): 0.2895
Accuracy (etu, val) : 0.3444 | Accuracy (dis, val) : 0.3512
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 386s 550ms/step - acc (etu, train): 0.3900 - acc (dis, train): 0.3990
Accuracy (etu, val) : 0.4008 | Accuracy (dis, val) : 0.4067
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 382s 543ms/step - acc (etu, train): 0.4487 - acc (dis, train): 0.4615
Accuracy (etu, val) : 0.4808 | Accuracy (dis, val) : 0.4898
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 390s 555ms/step - acc (etu, train): 0.4844 - acc (dis, train): 0.4972
Accuracy (etu, val) : 0.4734 | Accuracy (dis, val) : 0.4842
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 392s 558ms/step - acc (etu, train): 0.5197 - acc (dis, t

In [20]:
distillation_resnet18(3,0.5)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 417s 551ms/step - acc (etu, train): 0.0895 - acc (dis, train): 0.0913
Accuracy (etu, val) : 0.1234 | Accuracy (dis, val) : 0.1230
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 338s 480ms/step - acc (etu, train): 0.2496 - acc (dis, train): 0.2556
Accuracy (etu, val) : 0.3317 | Accuracy (dis, val) : 0.3385
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 332s 472ms/step - acc (etu, train): 0.3539 - acc (dis, train): 0.3666
Accuracy (etu, val) : 0.4052 | Accuracy (dis, val) : 0.4141
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 332s 473ms/step - acc (etu, train): 0.4236 - acc (dis, train): 0.4344
Accuracy (etu, val) : 0.4623 | Accuracy (dis, val) : 0.4722
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 330s 470ms/step - acc (etu, train): 0.4704 - acc (dis, train): 0.4820
Accuracy (etu, val) : 0.5108 | Accuracy (dis, val) : 0.5210
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 335s 476ms/step - acc (etu, train): 0.5076 - acc (dis, t

In [21]:
distillation_resnet18(8,0.5)

C'est parti pour la distillation !

Époque 1 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 466s 629ms/step - acc (etu, train): 0.0670 - acc (dis, train): 0.0688
Accuracy (etu, val) : 0.0641 | Accuracy (dis, val) : 0.0643
Époque 2 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 382s 544ms/step - acc (etu, train): 0.2025 - acc (dis, train): 0.2089
Accuracy (etu, val) : 0.2794 | Accuracy (dis, val) : 0.2817
Époque 3 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 376s 535ms/step - acc (etu, train): 0.3149 - acc (dis, train): 0.3217
Accuracy (etu, val) : 0.3736 | Accuracy (dis, val) : 0.3770
Époque 4 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 377s 537ms/step - acc (etu, train): 0.3902 - acc (dis, train): 0.4011
Accuracy (etu, val) : 0.4293 | Accuracy (dis, val) : 0.4321
Époque 5 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 384s 546ms/step - acc (etu, train): 0.4369 - acc (dis, train): 0.4477
Accuracy (etu, val) : 0.4726 | Accuracy (dis, val) : 0.4778
Époque 6 / 50
703/703 ━━━━━━━━━━━━━━━━━━━━ 380s 540ms/step - acc (etu, train): 0.4780 - acc (dis, t